<a href="https://colab.research.google.com/github/CompPsychology/psych290_colab_public/blob/main/notebooks/week-01/W1_Tutorial_01B_SQL_where_(album)_preHW1_withSolutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# W1 Tutorial 1B -- Intro to SQL, where onward (DB: album) (2025-03)

(c) Johannes Eichstaedt & the World Well-Being Project, 2023.

✋🏻✋🏻 NOTE - You need to create a copy of this notebook before you work through it. Click on "Save a copy in Drive" option in the File menu, and safe it to your Google Drive.

✉️🐞 If you find a bug/something doesn't work, please slack us a screenshot, or email johannes.courses@gmail.com.

Welcome to our second tutorial. Let's continue with the rest of the intro to SQL!

**FYI:** you can execute a cell by hitting `CTRL+Enter` (Win) or `Command+Enter` (Mac).   
`Shift+Enter` or `Command+Enter` will execute + advance to the cell below.

Please execute every cell as you go along.

**FYI:**
* 🤓🤓🤓 comparisons with the tidyverse are flagged with the triple nerd  
* 🐬🐬🐬 when there is code that runs in MySQL but not in SQLite, this is marked with the triple dolphin

## 1) Setting up Colab with DLATK and SQLite

We begin by setting up the Colab environment. The next couple of subsections do this for you.

You don't need to understand or follow along with the code -- it uses git (as in GitHub) to pull the code fromt the cloud and install it (with `pip`).

This will take ~1.5 to 2 minutes. If colab asks you about this not being authored by Google, say "Run anyway."

### 1a to 1c) Streamlined: Setting up Colab with DLATK and your data

In [ ]:
# enter your username!
username = "your_name"

########### 1a) Install

# installing DLATK and necessary packages
!git clone -b psych290 https://github.com/dlatk/dlatk.git
!pip install dlatk/
!pip install jupysql

########### 1b) Download data and insert into SQLite database
# We then load the downloaded data into a database named [username].db in the sqlite_data folder.

# this download the album csvs we need for this tutorial
!git clone https://github.com/CompPsychology/album.git

# load the required package -- similar to library() function in R
import os
from dlatk.tools.importmethods import csvToSQLite

# store the complete path to the database -- sqlite_data/[username].db
database = os.path.join("sqlite_data", username)

# import CSVs into tables in this database
csvToSQLite(
    "album/data/album.csv",
    database,
    "album"
)

csvToSQLite(
    "album/data/track.csv",
    database,
    "track"
)

############# 1c) Setup database connection
# Establish a connection with the (SQLite) database with the `%sql` extension from colab.

# loads the %%sql extension
%load_ext sql

# connects the extension to the database
from sqlalchemy import create_engine
engine = create_engine(f"sqlite:///sqlite_data/{username}.db?charset=utf8mb4")
%sql engine

#set the output limit to 50
%config SqlMagic.displaylimit = 50

## PRINT FINISHED
print(" ******* LOAD FINISHED ¯\_(ツ)_/ *******")

fatal: destination path 'dlatk' already exists and is not an empty directory.
Processing ./dlatk
  Preparing metadata (setup.py) ... done
  Created wheel for dlatk: filename=dlatk-1.3.1-py3-none-any.whl size=35635829 sha256=403345267d3e88f95c7b270816253125ce0ae129ffee6f53193c413f90b73921
  Stored in directory: /tmp/pip-ephem-wheel-cache-ubde0oy6/wheels/cc/c9/65/e1ecc64bac68518c07b286fe86921aa938e11a0c3a87d8ff93
Successfully built dlatk
  Attempting uninstall: dlatk
    Found existing installation: dlatk 1.3.1
    Uninstalling dlatk-1.3.1:
      Successfully uninstalled dlatk-1.3.1


fatal: destination path 'album' already exists and is not an empty directory.
The sql extension is already loaded. To reload it, use:
  %reload_ext sql
 ******* LOAD FINISHED ¯\_(ツ)_/ *******


Table already exists
Table already exists


## 2) Recap: SELECT Statement

Let's just remind ourselves what's in the tables.

In [ ]:
%sql PRAGMA table_info(album)

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

cid,name,type,notnull,dflt_value,pk
0,id,INT,0,None,0
1,title,VARCHAR(31),0,None,0
2,artist,VARCHAR(63),0,None,0
3,label,VARCHAR(15),0,None,0
4,released,VARCHAR(15),0,None,0


In [ ]:
%%sql

SELECT *
FROM album;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id,title,artist,label,released
1,Two Men with the Blues,Willie Nelson and Wynton Marsalis,Blue Note,2008-07-08
11,Hendrix in the West,Jimi Hendrix,Polydor,1972-01-01
12,Rubber Soul,The Beatles,Parlophone,1965-12-03
13,Birds of Fire,Mahavishnu Orchestra,Columbia,1973-03-01
16,Live And,Johnny Winter,Columbia,1971-05-01
17,Apostrophe,Frank Zappa,DiscReet,1974-04-22
18,Kind of Blue,Miles Davis,Columbia,1959-08-17


In [ ]:
%%sql

SELECT *
FROM track
ORDER BY duration DESC
limit 10;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id,album_id,title,track_number,duration
21,11,Red House,8,786
51,16,It's My Own Fault,2,734
68,18,All Blues,4,696
43,13,One Word,7,597
66,18,Freddy Freeloader,2,589
69,18,Flamenco Sketches,5,566
65,18,So What,1,565
70,11,Fake Track,9,549
54,16,Mean Town Blues,5,539
17,11,Voodoo Chile,4,469


## 3) WHERE Clause <a class="anchor" id="hs"></a>

Where were we? Right.

In this section, we will experiment with some approaches to select particular records/rows.

We use the **WHERE** clause to do this. **WHERE** is followed by a conditional expression, and any record that evaluates it to TRUE gets picked into the result set.

For example, we can **SELECT** the columns `title` and `label` **FROM** the table `album` **WHERE** `artist` is `The Beatles`, as shown below.

Basically, where let's you search among the rows.

As you can see, nothing here is record science. 👩‍🚀😉

In [ ]:
%%sql

SELECT title AS Title, label AS Label
FROM album
WHERE artist='The Beatles';

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

Title,Label
Rubber Soul,Parlophone


🤓🤓🤓 in the tidyverse, the WHERE is `filter`.

`album %>% select(artist, title, label) %>% filter(artist == "the Beatles")`

-- I actually think this wouldn't work, you'd have to include artist among the select variables. Same idea though.

Of course you can **WHERE** with anything else that can be evaluated by SQL.

#### 👩‍🔬💻 Exercise

For example,  can you pull out all the albums released after 1970?

**HINT :** Run `SELECT strftime('%Y', released) FROM album` first and see what that gives you. Muddle through from there.

Normally, we would do `YEAR(date)`, but sqlite doesn't have this, so we use the function `strftime('%Y', released)`.

The `strftime()` function can extract [a number of datetime features from a date](https://www.sqlitetutorial.net/sqlite-date-functions/sqlite-strftime-function/)  (e.g., year, month, day).


```
🐬🐬🐬
SELECT *
FROM album
WHERE YEAR(released) > 1970;
🐬🐬🐬
```


In [ ]:
%%sql

SELECT strftime('%Y', released) AS year
FROM album;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

year
2008
1972
1965
1973
1971
1974
1959


### 3a) Combining Conditional Expressions <a class="anchor" id="cce"></a>

As described in the above example, **WHERE** clause is followed by a conditional expression.

This expression is not restricted to a single condition but can be a complex expression that contains multiple simpler expressions chained using **AND** or **OR**, and evaluates to either **TRUE** or **FALSE**. #boolean

For example, below is set of queries to find the details of all 'The Beatles' songs that are longer than 2.5 minutes.

Let's first find out which id the album has was written by the Beatles:

In [ ]:
%%sql

SELECT artist, id
FROM album
WHERE artist='The Beatles';

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id
12


Now, let's select FROM only those tracks from the track people ONLY those that are 2.5 minutes (150 sec) or longer.

In [ ]:
%%sql

SELECT *
FROM track
WHERE (duration > 150) AND (album_id = 12);

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id,album_id,title,track_number,duration
24,12,You Won't See Me,3,202
25,12,Nowhere Man,4,164
27,12,The Word,6,163
28,12,Michelle,7,162
29,12,What Goes On,8,170
30,12,Girl,9,153


Ok, great.

### 3b) Combining Conditional Expressions II

Ok, let's try to find all the Beatles albums released before 1975 in one query from the album table?

Let's look at that table.

In [ ]:
%%sql

SELECT *
FROM album;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id,title,artist,label,released
1,Two Men with the Blues,Willie Nelson and Wynton Marsalis,Blue Note,2008-07-08
11,Hendrix in the West,Jimi Hendrix,Polydor,1972-01-01
12,Rubber Soul,The Beatles,Parlophone,1965-12-03
13,Birds of Fire,Mahavishnu Orchestra,Columbia,1973-03-01
16,Live And,Johnny Winter,Columbia,1971-05-01
17,Apostrophe,Frank Zappa,DiscReet,1974-04-22
18,Kind of Blue,Miles Davis,Columbia,1959-08-17


🐇 this is slightly more advanced: DATES in SQL are encoded using the YYYY-MM-DD format. If you only want to get the YYYY from it, you can use `strftime('%Y', released)` function around such a column:

In [ ]:
%%sql

SELECT title, strftime('%Y', released) AS year

FROM album;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

title,year
Two Men with the Blues,2008
Hendrix in the West,1972
Rubber Soul,1965
Birds of Fire,1973
Live And,1971
Apostrophe,1974
Kind of Blue,1959


This is equivalent to YEAR() in MySQL

🐬🐬🐬
```
SELECT title, YEAR(released)
FROM album
```
🐬🐬🐬

#### 👩‍🔬💻 Exercise

OK, can you know put these tricks together to find the all the Beatles albums released before 1975.

In [ ]:
%%sql

SELECT *
FROM album
WHERE (artist = 'The Beatles') AND (strftime('%Y', released) < "1975");

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id,title,artist,label,released
12,Rubber Soul,The Beatles,Parlophone,1965-12-03


🐬🐬🐬
```
SELECT *
FROM album
WHERE (artist = 'The Beatles') AND (YEAR(released) < 1975);
```
🐬🐬🐬

### 3c) Pattern Matching <a class="anchor" id="pm"></a>

🤔 But how do we get rows that have some pattern in their text?

Unlike the overused verbal non-fluency, the SQL **LIKE** keyword helps us to do that. **LIKE** provides a regular expression based pattern matching to retrieve records from a table. For example, the below command helps us retrieve album titles that begin with 'A'.

In [ ]:
%%sql

SELECT *
FROM album
WHERE title LIKE 'A%';

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id,title,artist,label,released
17,Apostrophe,Frank Zappa,DiscReet,1974-04-22


`%` is a wildcard that matches **any character and any number of them**. So, in the above query *'A%'* matches a string that has *A* as its first character and some characters after that. An extensive demonstration of all the wildcards is beyond the scope of this tutorial (and maybe even the course). You can always check them in the [documentation](https://dev.mysql.com/doc/refman/8.0/en/pattern-matching.html).

You generally can get by knowing just this one `%` wildcard.

🤓🤓🤓 in the tidyverse, this would be (with `str_detect` from the stringr package)

`album %>% filter(str_detect(title,"^A"))`

(the ^ means "starts with" in regular expressions)

As you will have realized by now, we can use the same *%* wildcard to get records which have columns ending in a certain pattern. Let's say we want to find a record where the artist has last name `Hendrix`.

In [ ]:
%%sql

SELECT *
FROM album
WHERE artist LIKE '%Hendrix';

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id,title,artist,label,released
11,Hendrix in the West,Jimi Hendrix,Polydor,1972-01-01


The use case `LIKE '%Hendrix%'` is probably most useful -- find anything that contains the string. 0 characters before or after are also a [number of characters](https://www.youtube.com/watch?v=UfJljXugLA0), and will also be matched by `%`.

#### 👩‍🔬💻 Exercise

Ok, now you try... which tracks contain `Own`? Which album labels contain `Poly`?

In [ ]:
%%sql

SELECT *
FROM track
WHERE title LIKE "%Own%";

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id,album_id,title,track_number,duration
23,12,Norwegian Wood (This Bird Has Flown),2,125
51,16,It's My Own Fault,2,734
54,16,Mean Town Blues,5,539


In [ ]:
%%sql

SELECT *
FROM album
WHERE label LIKE "%Poly%";

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id,title,artist,label,released
11,Hendrix in the West,Jimi Hendrix,Polydor,1972-01-01


Somewhat less useful, but good to know, is the `_` wildcard.

Unlike `%` which matches an arbitrary number of characters, `_` (underscore)  matches a single character. This helps particularly when you know the number of characters in a pattern. Let's say we have to find the record which has 6 or more letters in the last word of the artist name.

(Notice the trick with requiring the space in the middle of the artist name.)

In [ ]:
%%sql

SELECT *
FROM album
WHERE artist LIKE '% ______%';

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id,title,artist,label,released
1,Two Men with the Blues,Willie Nelson and Wynton Marsalis,Blue Note,2008-07-08
11,Hendrix in the West,Jimi Hendrix,Polydor,1972-01-01
12,Rubber Soul,The Beatles,Parlophone,1965-12-03
13,Birds of Fire,Mahavishnu Orchestra,Columbia,1973-03-01
16,Live And,Johnny Winter,Columbia,1971-05-01


Here `%` initially matches any number of characters in the words before the last one. Then ' ' matches the whitespace. This is then followed by a series of 5 `_`'s that match to the minimum number of characters needed and `%` then matches the rest (including zero charecters). So any lastname with 5+ chars is eligible.

As you can see SQL is pretty simple, but understanding it deeply allows you to do most anything. That is it's power. That's why it's the basis of data science 🎓.

#### 👩‍🔬💻 Exercise

Alright, show the tracks that start with a three letter word (technically, a 3-char token. what are the odds??)*

In [ ]:
%%sql


SELECT *
FROM track
WHERE title LIKE "___ %";

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id,album_id,title,track_number,duration
18,11,The Queen,5,160
21,11,Red House,8,786
24,12,You Won't See Me,3,202
27,12,The Word,6,163
31,12,I'm Looking Through You,10,147
35,12,Run for Your Life,14,138
43,13,One Word,7,597
58,17,St. Alfonzo's Pancake Breakfast,3,110
68,18,All Blues,4,696


*(this is a joke, based on fundamental laws of language use. Short words are highly frequent. Hahaha.)

## 4) HAVING Clause <a class="anchor" id="having"></a>

`HAVING` is basically another `WHERE` that comes at the very end of the query -- a final filter. You can use it to filter down the output of `group by` clauses, which you couldn't do with a `WHERE` -- that runs before the `group by`, logically. We are getting fancy-pants, admittedly.

It augments the **GROUP BY** clause by considering particular groups into the result set that satisfy certain conditions. Consider the query to find the average duration of tracks (grouped by *album_id*) for a particular album.

In [ ]:
%%sql

SELECT album_id, AVG(duration) AS average
FROM track
GROUP BY album_id;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

album_id,average
1,320.7
11,330.0
12,152.78571428571428
13,242.4
16,405.3333333333333
17,211.88888888888889
18,550.8


We see that some of the albums have an average duration of more than 5 minutes. Let's shortlist down to those!

In [ ]:
%%sql

SELECT album_id, AVG(duration) AS average_duration
FROM track
GROUP BY album_id
HAVING average_duration > 300;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

album_id,average_duration
1,320.7
11,330.0
16,405.3333333333333
18,550.8


🤓🤓🤓 in the tidyverse, this would be

`track %>% select(album_id, duration) %>% group_by(album_id) %>%
    summarize(average_duration = mean(duration)) %>% filter(average_duration > 300)`
    
As you can see, we are getting into deeper waters.

We can also mix different functions in the aggregation and the having :
    
**HAVING** allows us to find average durations of more than 5 minutes, and then we could apply the **SUM** function to such group.


In [ ]:
%%sql

SELECT album_id, SUM(duration) AS total_duration
FROM track
GROUP BY album_id
HAVING AVG(duration) > 300;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

album_id,total_duration
1,3207
11,2970
16,2432
18,2754


#### 👩‍🔬💻 Exercise

Can you write the dplyr for the above command? Skip if this is not exciting to you, etc.

```
track %>%
    group_by(album_id) %>%
    mutate(total_duration=sum(duration), avg_duration=mean(duration) %>%
    filter(avg_duration > 300) %>%
    select(album_id, total_duration)
```

#### 👩‍🔬💻 Exercise

How about a SELECT that shows all albums with a duration range (`max` minus `min`) larger than a 100 seconds?

In [ ]:
%%sql

SELECT album_id
FROM track
GROUP BY album_id
HAVING (MAX(duration) - MIN(duration)) > 100;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

album_id
1
11
13
16
17
18


## 5) JOIN <a class="anchor" id="join"></a>

Until now, we used a single table as argument for all our queries. In fact, we even followed a multi-step process for a certain result above while it could've been clearly done in a single query involving both the tables.

The **JOIN** clause helps us do that. As the name conveys, it helps join multiple tables and extract information from them as shown below.

⚠️ **DISCLAIMER: NEVER USE A JOIN CLAUSE WITHOUT SPECIFYING WHAT TO JOIN ON.** SQL would just join every row on every other row, and the whole thing blows up (gets slow and then crashes, depending on how big the tables are.)

We already know that the two tables are connected with a primary key - foreign key relationship (`id` in `album` is `album_id` in `track`), we can simply filter the records based on the availability of foreign key of one table as the primary key of the other using the **ON** clause as shown below.

In [ ]:
%%sql

SELECT *
FROM album JOIN track
ON album.id=track.album_id
LIMIT 5;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id,title,artist,label,released,id_1,album_id,title_1,track_number,duration
1,Two Men with the Blues,Willie Nelson and Wynton Marsalis,Blue Note,2008-07-08,1,1,Bright Lights Big City,1,320
1,Two Men with the Blues,Willie Nelson and Wynton Marsalis,Blue Note,2008-07-08,2,1,Night Life,2,344
1,Two Men with the Blues,Willie Nelson and Wynton Marsalis,Blue Note,2008-07-08,3,1,Basin Street Blues,5,296
1,Two Men with the Blues,Willie Nelson and Wynton Marsalis,Blue Note,2008-07-08,4,1,Caldonia,3,205
1,Two Men with the Blues,Willie Nelson and Wynton Marsalis,Blue Note,2008-07-08,5,1,Stardust,4,308


😍 This is really where SQL shines, and what it's designed to do -- compose output on the fly from multiple tables. You don't store it all joined because that's just a huge waste of space. You pull things together on the fly.

Technically, this an **INNER JOIN**, as it only returns rows that have ids in both tables. There is also a **LEFT JOIN** (return all from left and only matching from right), **RIGHT JOIN** and **OUTER JOIN**. Every tutorial teaches these, I've used them twice in 10 years. Let's ignore them. But if nothing else, it will clue you in where the tidyverse verbs come from!


🤓🤓🤓 In the tidyverse, we would do this like so

`inner_join(album, track, by.x = "id", by.y = "album_id")` -- alternatively, `merge()`

**NOTE**
⚠️⚠️⚠️
`TABLE.VARIABLE` (with a period in between) just helps us to specify the table to which the `id` column belongs to as both the tables contain one and track ID is of no concern to us. This is an important thing to know when you work with multiple tables -- rather than `select variable` you can ALWAYS say `SELECT table.variable` and even `SELECT database.table.variable` to avoid all confusion, or to select from multiple databases at once when you have tables in multiple databases

Addressing our long belabored task from the above multi step process, below is the query to extract the details of all 'The Beatles' song that are longer than 2.5 minutes. There are multiple ways to achieve this and we will begin demonstrating it with the simplest method.

Hint: this is the dplyr for this:
        
```
merge(album, track, by.x='id', by.y='album_id') %>%
    filter((artist == 'The Beatles') & (duration > 150))
```

In [ ]:
%%sql

SELECT *
FROM album JOIN track
ON (album.id=track.album_id)
WHERE (album.artist='The Beatles') AND (track.duration > 150);

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id,title,artist,label,released,id_1,album_id,title_1,track_number,duration
12,Rubber Soul,The Beatles,Parlophone,1965-12-03,24,12,You Won't See Me,3,202
12,Rubber Soul,The Beatles,Parlophone,1965-12-03,25,12,Nowhere Man,4,164
12,Rubber Soul,The Beatles,Parlophone,1965-12-03,27,12,The Word,6,163
12,Rubber Soul,The Beatles,Parlophone,1965-12-03,28,12,Michelle,7,162
12,Rubber Soul,The Beatles,Parlophone,1965-12-03,29,12,What Goes On,8,170
12,Rubber Soul,The Beatles,Parlophone,1965-12-03,30,12,Girl,9,153


#### 👩‍🔬💻 Exercise

Alright there, buddy friend. How about a query that shows only the albums that have a track with number 8?

In [ ]:
%%sql

SELECT DISTINCT track.album_id
FROM album JOIN track
ON (album.id=track.album_id)
WHERE track.track_number = 8;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

album_id
1
11
12
13
17


#### 👩‍🔬💻 Exercise

Show tracks with artists who start with "The" who were released before 1970?

**Hint:** `artist` and `released` is in album, tracks are in `track`.  

In [ ]:
%%sql

SELECT *
FROM album JOIN track
ON (album.id=track.album_id)
WHERE (strftime("%Y", album.released) < "1970") AND (album.artist LIKE "The%");

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id,title,artist,label,released,id_1,album_id,title_1,track_number,duration
12,Rubber Soul,The Beatles,Parlophone,1965-12-03,22,12,Drive My Car,1,150
12,Rubber Soul,The Beatles,Parlophone,1965-12-03,23,12,Norwegian Wood (This Bird Has Flown),2,125
12,Rubber Soul,The Beatles,Parlophone,1965-12-03,24,12,You Won't See Me,3,202
12,Rubber Soul,The Beatles,Parlophone,1965-12-03,25,12,Nowhere Man,4,164
12,Rubber Soul,The Beatles,Parlophone,1965-12-03,26,12,Think for Yourself,5,139
12,Rubber Soul,The Beatles,Parlophone,1965-12-03,27,12,The Word,6,163
12,Rubber Soul,The Beatles,Parlophone,1965-12-03,28,12,Michelle,7,162
12,Rubber Soul,The Beatles,Parlophone,1965-12-03,29,12,What Goes On,8,170
12,Rubber Soul,The Beatles,Parlophone,1965-12-03,30,12,Girl,9,153
12,Rubber Soul,The Beatles,Parlophone,1965-12-03,31,12,I'm Looking Through You,10,147


## 6) Implicit Join <a class="anchor" id="ijoin"></a>

For all our purposes, implicit join is same as the join discussed above.

Instead of the **JOIN** keyword we can simply list the tables separated by commas and the condition under the **ON** clause is included in **WHERE** with an **AND**. Here, the comma "implicitly" conveys the join of the tables listed. Let's repeat the above query using implicit join. Going forward, we will use implicit join to shorten the queries and also to make them more intuitive.

But make sure you are tracking what we are doing, we are taking the training wheels off here.

In [ ]:
%%sql

SELECT *
FROM album, track
WHERE (album.id=track.album_id) AND (album.artist='The Beatles') AND (track.duration > 180);

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id,title,artist,label,released,id_1,album_id,title_1,track_number,duration
12,Rubber Soul,The Beatles,Parlophone,1965-12-03,24,12,You Won't See Me,3,202


😍 Now SQL is beginning to become notably more efficient than dplyr.

#### 👩‍🔬💻 Exercise

Alright, do "only the albums that have a track with number 8" with an implicit join.

In [ ]:
%%sql

SELECT DISTINCT track.album_id
FROM album, track
WHERE (album.id=track.album_id) AND (track.track_number = 8);

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

album_id
1
11
12
13
17


Time for another break!! There is only so much you can absorb per unit human time 🤖.

## 7) Subqueries <a class="anchor" id="sq"></a>

Alright, time for the big boy pants (or girl or they pants).

We can also achieve the same using a "subquery". Subqueries are the queries that are part of another query which consumes the result returned by the subqueries. We can use such subqueries in the **FROM** clause or the **WHERE**, as demonstrated below.

Basically, you just make yourself a table on the fly mid-query with a nested **SELECT**.

Sounds complicated. Also is complicated.

Here's an example where we achieve the above result by using a subquery in the **FROM** clause. We initially pick the necessary columns from the rows in the *track* table that have duration greater than 3 minutes. The result of the subquery (another table) is used to then filter out the details of only tracks sung by `The Beatles`.

so first we do:

In [ ]:
%%sql

SELECT album_id, title, track_number, duration
FROM track
WHERE duration > 180;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

album_id,title,track_number,duration
1,Bright Lights Big City,1,320
1,Night Life,2,344
1,Basin Street Blues,5,296
1,Caldonia,3,205
1,Stardust,4,308
1,Georgia On My Mind,6,280
1,Rainy Day Blues,7,343
1,My Bucket's Got A Hole In It,8,296
1,Ain't Nobody's Business,9,447
1,That's All,10,368


but we nest this in on the fly:

In [ ]:
%%sql

SELECT *
FROM album, (
  SELECT album_id, title, track_number, duration
  FROM track
  WHERE duration > 180) AS filtered_table
WHERE (album.id=filtered_table.album_id) AND (album.artist='The Beatles');

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id,title,artist,label,released,album_id,title_1,track_number,duration
12,Rubber Soul,The Beatles,Parlophone,1965-12-03,12,You Won't See Me,3,202


**WHERE** with the **IN** keyword

We can obtain the same results by using the idea of subquery in the `WHERE` clause as shown below. Here the subquery obtains the album id for any album by *The Beatles*, and the outer query checks if the track has it's album id **IN** the subquery results and also if its duration is more than 3 minutes. Note the use of **IN** keyword there.

We will also use **JOIN** keyword just for recollection, feel free to use implicit join.

In [ ]:
%%sql

SELECT *
FROM album JOIN track
ON (album.id=track.album_id)
WHERE (track.duration>180) AND (track.album_id IN (SELECT id FROM album WHERE artist='The Beatles'));

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id,title,artist,label,released,id_1,album_id,title_1,track_number,duration
12,Rubber Soul,The Beatles,Parlophone,1965-12-03,24,12,You Won't See Me,3,202


As demonstrated above, you have multiple options to obtain the same results.

Honestly, I rarely use subqueries, unless I really have to. But sometimes that's the only way. Some poeple love them.

Great way to impress your new coworkers in tech!

## 8) Putting it all together 🕺💃

Now, let's find the average duration of tracks for every label. Remember that the details of tracks and of labels are in different tables, and we need to **JOIN** them. We'll then **GROUP BY** and **AVG** the resulting table. This query includes multiple ideas explained until now thus demonstrating the flexibility of MySQL.

Subqueries always get developed step by step. So let's get a list track durations for different labels.

In [ ]:
%%sql

SELECT label, duration
FROM album JOIN track ON album.id=track.album_id

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

label,duration
Blue Note,205
Blue Note,280
Blue Note,296
Blue Note,296
Blue Note,308
Blue Note,320
Blue Note,343
Blue Note,344
Blue Note,368
Blue Note,447


Ok, now let's use that as a subquery (we select FROM it), and AVG-GROUP BY label to get the average durations.

In [ ]:
%%sql

SELECT label, AVG(duration)
FROM (SELECT label, duration
      FROM album JOIN track
      ON album.id=track.album_id) AS joined
GROUP BY label;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

label,AVG(duration)
Blue Note,320.7
Columbia,362.3809523809524
DiscReet,211.88888888888889
Parlophone,152.78571428571428
Polydor,330.0


nice, ehh?

#### 👩‍🔬💻 Exercise

Can you do this with an implicit **WHERE** join instead?

In [ ]:
%%sql

SELECT label, AVG(duration)
FROM album, track
WHERE album.id=track.album_id
GROUP BY label;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

label,AVG(duration)
Blue Note,320.7
Columbia,362.3809523809524
DiscReet,211.88888888888889
Parlophone,152.78571428571428
Polydor,330.0


#### 👩‍🔬💻 Exercise

Let's do one more, but this time we will include **HAVING** clause as well.

Let's find the average duration of tracks for labels with atleast 10 tracks in them.

In [ ]:
%%sql

SELECT label, AVG(duration) AS average, COUNT(*) AS count
FROM (SELECT label, duration
      FROM album JOIN track
      ON album.id=track.album_id) AS joined
GROUP BY label
HAVING count >= 10;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

label,average,count
Blue Note,320.7,10
Columbia,362.3809523809524,21
Parlophone,152.78571428571428,14


## 9) CREATE <a class="anchor" id="create"></a>

OK, let's do a little more on how to create tables themselves.

**CREATE** statement can be used to create both databases and tables (pretty similar to functionality of 🐬🐬🐬**SHOW** statement in MySQL). We will again restrict ourselves to examples that are necessary for us in this course. Hence, we will particularly discuss 2 methods to create a table.

### 9a) CREATE TABLE ... AS SELECT ... <a class="anchor" id="cts"></a>

This command is an absolute killer feature.

Save the output of a SELECT query into a new table -- very helpful if you want to break longer flows into steps that you can quality control, and avoid driving yourself insane with subqueries 🤪.

We use the **CREATE TABLE ... SELECT** statement then, which creates a new table and fills it with the result of the **SELECT** statement. Let's say we have to create another table `duplicate_album` though which is identical to `album`.

In [ ]:
%%sql

CREATE TABLE duplicate_album AS
SELECT * FROM album;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

++
||
++
++

🤓🤓🤓 tidyverse

`duplicate_album <- album`

Btw, if you try to write a table that already exists, you get an error (try the above a few times.)

You can overwrite the prior table if it's there:

In [ ]:
%%sql

DROP TABLE IF EXISTS duplicate_album;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

++
||
++
++

#### 👩‍🔬💻 Exercise

Can you make a short-durations track table, say, less than 3 minutes?

In [ ]:
%%sql

DROP TABLE IF EXISTS short_tracks;
CREATE TABLE short_tracks AS SELECT album_id, title, track_number, duration FROM track WHERE duration <= 180;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

++
||
++
++

⚠️ This is a handy and fast way to make tables, but they won't have indices on them (see below).

* recommended: just add indices manually aftewrards
* In MySQL, you could also do a combo of 🐬🐬🐬`CREATE TABLE newtable LIKE` and `INSERT INTO newtable SELECT your_other_query` -- this would keep indices. But now we are getting into the advanced weeds. But that's what I do week-to-week. It's two commands, but you don't have to worry about indices.

I realized this tutorial hasn't talked about indices yet. They are very very important.

### 9b) 🐬🐬🐬 CREATE TABLE ... LIKE <a class="anchor" id="ctl"></a>

🐬🐬🐬 **CREATE TABLE ... LIKE** is used to create an empty table in the "form" of another table in MySQL. For example, let's create another table *duplicate_track* though empty but similar to *track*.

Though this isn't how you'll create tables in Colab, it's a very useful feature in MySQL.

```
CREATE TABLE duplicate_track AS track;
```

```
🐬🐬🐬
SELECT *
FROM duplicate_track;
🐬🐬🐬
```

```
🐬🐬🐬
DESCRIBE duplicate_track;
🐬🐬🐬
```

As you can observe, we have an empty table `duplicate_track` with the same columns as `track` and **also the same primary key and indices**. This method is useful when we have to create a similar table although with different content to be inserted later -- we can just "template" one table like the other.

## 10) ALTER TABLE Statement <a class="anchor" id="alter"></a>

**ALTER** and **UPDATE** are the 2 statements majorly used to alter/update a database. **ALTER** is used to alter the structure of a database, table, functions, etc. We rarely perform such operations other than updating index of a table (in 🐬🐬🐬 MySQL). We'll discuss another way to update an index in this notebook.

Whereas **UPDATE** is used to modify the rows in a table mostly based on some conditions, similar to a **SELECT** statement. A more extensive discussion of these statements can be found at https://dev.mysql.com/doc/refman/8.0/en/sql-statements.html section 13.1 and 13.2.

The below **PRAGMA** statement displays the properties of columns in *track* table. We can observe that *id* column is the primary key -- it uniquely identifies each row.

In [ ]:
%%sql

PRAGMA table_info(track);

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

cid,name,type,notnull,dflt_value,pk
0,id,INT,0,None,0
1,album_id,INT,0,None,0
2,title,VARCHAR(63),0,None,0
3,track_number,INT,0,None,0
4,duration,INT,0,None,0


In [ ]:
%%sql

SELECT id from track;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id
1
2
3
4
5
6
7
8
9
10


## 11) Creating Indexes

🤓 This is really important, even though it's all the way down there at the end. Indexes make or break a database.

There are queries which are executed frequently based on the use case.

For example, we want to retrieve the list of students in alphabetical order of their SUNetIDs. Then repeating calling of **SELECT ... ORDER BY** would internally require sorting the rows based on SUNetID which is very resource and time intensive, particularly if we need to do this frequently.

This situation also exists when we have to search for a student using SUNetID, we can't simply iterate through the rows due to their number. Such cases call for optimization of these queries for faster retrievel.

To **optimize** these queries, we can add an index to the respective columns for faster retrieval. Though the query results remain the same, SQL flavors store an additional data structure internally to retrieve these columns faster.

For example, we can add an index to the `album_id` column, we can do that as follows. The syntax is a little clunky -- you tell it what column the index goes on, and then what you want to call the index (just call it the same as the column 🙄).

**Important**: The standard way to create an index is to use `idx_tablename_colname`. For example, `idx_track_album_id`.

In [ ]:
%%sql

CREATE INDEX idx_track_album_id
ON track(album_id);

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

++
||
++
++

Let's also make an index on id in album, so we can join them on the two indices!

In [ ]:
%%sql

CREATE INDEX idx_album_id
ON album(id);

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

++
||
++
++

We can see the added index using **`PRAGMA index_list()`** (or 🐬🐬🐬 SHOW in MySQL) statement as follows.

In [ ]:
%%sql

PRAGMA index_list(album)

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

seq,name,unique,origin,partial
0,idx_album_id,0,c,0


Now if we wanted to search for a particular album name, it would be WAAAAAAY faster.

**⚠️⚠️⚠️ In real life ALWAYS HAVE INDICES ON COLUMNS YOU `JOIN` ON, and that you use IN YOUR `WHERE` CLAUSE.**

DLATK will create indices for you on feature tables it creates for you. But as you work with your own tables, you will forget. And things will take minutes when they could take seconds, with an index. Everybody always forgets. And waits.

### 11a) More INDEX practice <a class="anchor" id="uindex"></a>

Let's create a new table similar to `track` but containing only the `album_id` = 1, for this example. This results in `track_number` uniquely identify the rows in the new table.

**Note:** the command below makes the table, then looks at the table. SQL commands just need to be terminated with the `;`.

In [ ]:
%%sql

DROP TABLE IF EXISTS new_track;

CREATE TABLE new_track AS
SELECT * FROM track WHERE album_id=1;
SELECT * FROM new_track;

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

id,album_id,title,track_number,duration
1,1,Bright Lights Big City,1,320
2,1,Night Life,2,344
3,1,Basin Street Blues,5,296
4,1,Caldonia,3,205
5,1,Stardust,4,308
6,1,Georgia On My Mind,6,280
7,1,Rainy Day Blues,7,343
8,1,My Bucket's Got A Hole In It,8,296
9,1,Ain't Nobody's Business,9,447
10,1,That's All,10,368


Note that for these records, track_number is unique.

Let's put an index on `track_number` using `CREATE INDEX`.

In 🐬🐬🐬 MySQL we would use **ALTER** statement and **ADD INDEX**.

In [ ]:
%%sql

CREATE INDEX idx_track_track_number
ON new_track(track_number);

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

++
||
++
++

In [ ]:
%%sql

PRAGMA index_list(new_track);

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

seq,name,unique,origin,partial
0,idx_track_track_number,0,c,0


#### 👩‍🔬💻 Exercise

Can you make a new table that only contains tracks from "Miles Davis", and put an index  on the track number?

In [ ]:
%%sql

DROP TABLE IF EXISTS miles_album;

CREATE TABLE miles_album AS
    SELECT track.*
    FROM track, album
    WHERE track.album_id = album.id AND album.artist = "Miles Davis";

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

++
||
++
++

In [ ]:
%%sql

CREATE INDEX idx_miles_album_track_number
ON miles_album(track_number);

Running query in 'sqlite:///sqlite_data/your_name.db?charset=utf8mb4'

++
||
++
++

🌤🎉🎊 You've made it!! You should be ready now for the Homework 1. It won't be hard.

## -- TUTORIAL 1 END --